In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium as flm

world_map_ovr = flm.Map() # whole world map 
# define the world map centered around Canada (with coord)
world_map = flm.Map(location=[56.130, -106.35], zoom_start=4, tiles = 'Stamen Terrain')
# tiles are optional. 'Stamen Toner' for Black and White Maps, 'Stamen Terrain' for terrain maps

world_map

In [9]:
df_incidents = pd.read_csv("Police Incidents Data 2016.csv")
limit = 250
df_incidents = df_incidents.iloc[0:limit, :]

# San Francisco latitude and longitude values
latitude = 37.77
longitude = -122.42

# create map
sf_map = flm.Map(location=[latitude, longitude], zoom_start=12)

# loop through the 250 crimes and add each to the map
for lat, lng, label in zip(df_incidents.Y, df_incidents.X, df_incidents.Category):
    flm.CircleMarker(
        [lat, lng],
        radius=5, # define how big you want the circle markers to be
        color='yellow',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(sf_map)

latitudes = list(df_incidents.Y)
longitudes = list(df_incidents.X)
labels = list(df_incidents.Category)

for lat, lng, label in zip(latitudes, longitudes, labels):
    flm.Marker([lat, lng], popup=label).add_to(sf_map) 

# or can delete the label by only showing circular yellow, in the for loop 250 crimes add
# popup=label (after fill=)


sf_map

In [ ]:
from folium import plugins

# let's start again with a clean copy of the map of San Francisco
sf_map = flm.Map(location = [latitude, longitude], zoom_start = 12)

# create mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(sf_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(df_incidents.Y, df_incidents.X, df_incidents.Category):
    flm.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)

# display map
sf_map

In [ ]:
df_can = pd.read_excel("Canada.xlsx", sheet_name = "Canada by Citizenship", skiprows = range(20), skipfooter = 2)
df_can.drop(['AREA','REG','DEV','Type','Coverage'], axis=1, inplace=True)
df_can.rename(columns={'OdName':'Country', 'AreaName':'Continent','RegName':'Region'}, inplace=True)
df_can.columns = list(map(str, df_can.columns))
df_can['Total'] = df_can.sum(axis=1)
years = list(map(str, range(1980, 2014)))

import json
with open('world_countries') as myfile:
    world_geo = myfile.read()

# create choropleth world map
world_map = flm.Map(location=[0, 0], zoom_start=2)

world_map.choropleth(
    geo_data=world_geo,
    data=df_can,
    columns=['Country', 'Total'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Immigration to Canada'
)

world_map

#if need thresholding for legend:
#threshold_scale = np.linspace(dc['count'].min(),
#                              dc['count'].max(),
#                              6, dtype=int)
#threshold_scale = threshold_scale.tolist() # change the numpy array to a list
#threshold_scale[-1] = threshold_scale[-1] + 1 # make sure that the last value of the list is greater than the maximum immigration
#    and add  threshold_scale=threshold_scale  as parameter in choropleth, after key_on

In [11]:
df_incidents = pd.read_csv("Police Incidents Data 2016.csv")
df_sfnew = pd.DataFrame(df_incidents['PdDistrict'].value_counts()).reset_index()
df_sfnew = df_sfnew.rename(columns={'index':'Neighborhood', 'PdDistrict':'Count'})
df_sfnew.head()

,Neighborhood,Count
0,SOUTHERN,28445
1,NORTHERN,20100
2,MISSION,19503
3,CENTRAL,17666
4,BAYVIEW,14303


In [ ]:
sf_geo = 'san-francisco.geojson'


# create a plain world map
sf_map = flm.Map(location=[37.77, -122.42], zoom_start=12)
sf_map.choropleth(
    geo_data=sf_geo,
    data=df_sfnew,
    columns=['Neighborhood', 'Count'],
    key_on='feature.properties.DISTRICT',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Crimes in  San Francisco',
    reset=True
)
sf_map